<a href="https://colab.research.google.com/github/OnroerendErfgoed/scriptorium/blob/main/notebooks/01_get_resource.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

#Accessing a resource

Accessing a REST resource through Python is very easy when using the [Requests
library](https://docs.python-requests.org/en/latest/). It's a very flexible library that offers tremendous value for money. It's well worth spending some time with the documentation. These notebooks will show you the basics, but there is so much more out there.

The basic idea that we'll be using over and over consists of making a request and reading data from the response. In these scenarios we'll call our script the client where the requests originate. The response that we receive are sent by the server. 

Reponses can contain data in lots of different formats, but we'll mainly focus on JSON responses. These can be recognised by the `application/json` mime-type that is present in the `Content-Type` header of the response. This `Content-Type` is very important since it informs the client how it should try to interpret a certain resource. Flanders Heritage resources generally have at least two representations:
* `application/json`: The resources serialised in a machine-readable format. Suited for scripts, wrangling data, other servers, ...
* `text/html`: The resource serialised as HTML. Suited for your browser so you can read the resources when browsing the web.

From time to time you might encounter other mime types when using Flanders Heritage webservices such as `image/jpeg`, `image/png`, `application/rdf+xml`, `text/turtle`. Where applicable they will be explained.

In [ ]:
# Import requests library to contact the REST service
import requests
# Import JSON library to pretty print JSON
import json

# Make a request and store the response
response = requests.get(
    'https://id.erfgoed.net/erfgoedobjecten/200498',
    headers = {'Accept': 'application/json'}
)

# Turn the response's JSON data into a Python dictionary
data = response.json()

# Print the response's JSON data
print(json.dumps(data, sort_keys=True, indent=4))

This example ask for erfgoedobject 200498, the [Red Star Line buildings](https://inventaris.onroerenderfgoed.be/erfgoedobjecten/200498). It returns all data available on this object.

The Inventaris makes a distinction between three main categories or types of objects:
* **Erfgoedobjecten** (heritage objects): Parts of our heritage, seen in a way that makes sense to humans. By definition somewhat vague.
* **Aanduidingsobjecten** (designation objects): Pars of our heritage, but seen in a legal way. Heritage as it is known to law. By definition very precise.
* **Waarnemingen** (observations): Observations concering heritage. Things that were seen, but they might not necessary (still) have heritage value. Mainly used for archaological information.

More information about the [distinction between Erfgoedobject and Aanduidingsobject](https://onroerenderfgoed.github.io/posts/2019/inventaris-erfgoedobjecten-aanduidingsobjecten/). We will focus on the first two, Erfgoedobjecten and Aanduidingsobjecten, and the relations between them. These relations indicate what the legal status is of a certain Erfgoedobject. To detect these, we look at the **relations** attribute of the Erfgoedobject. There are several types of relations, but we are only interested in the ones with Aanduidingsobjecten. These can be detected by looking at the attribute **verwant** of every relation. We only want the ones with **id** 5.

Because scrolling through the whole object all the time (as we did in our first script) is rather unpractical, we'll limit ourself to only printing some basic information about an object.

In [ ]:
# Import requests library to contact the REST service
import requests
# Import JSON library to pretty print JSON
import json

# Make a request and store the response
response = requests.get(
    'https://id.erfgoed.net/erfgoedobjecten/200498',
    headers = {'Accept': 'application/json'}
)

# Turn the response's JSON data into a Python dictionary
data = response.json()

# Print URI and name of the Erfgoedobject
print(f"{data['uri']}: {data['naam']}")
# Print a short description of an object
print(f"{data['korte_beschrijving']}")

print(f"Designations:")
# Loop through all the relations
for r in data['relaties']:
  # Only use the relations with aanduidingsobjecten
  if r['verwant']['id'] == 5:
    # Print the type of relation, type of designation, URI of the object, name of the object
    print(f"{r['relatietype']['naam']} {r['aanduidingsobjecttype']} {r['uri']}: {r['omschrijving']}")

In this example, we only used data that was present in the object we requested. But there comes a point when we need some information not present in this object. In that case, we have to request another object that does have the information we need.

Suppose we're interested in the [Deecrees (Besluiten)](https://besluiten.onroerenderfgoed.be/) that led to the designation of a certain object. These are linked to the Aanduidingsobjecten, but in the previous examples we couldn't see them. When we request the Erfgoedobject, we get a summary of every coupled Aanduidingsobject, but not the full object. The full object does contain a link to the Besluit. In our next example we'll fetch the full Aanduidingsobject so we can access the Besluit.

In [ ]:
# Import requests library to contact the REST service
import requests
# Import JSON library to pretty print JSON
import json

# Make a request and store the response
response = requests.get(
    'https://id.erfgoed.net/erfgoedobjecten/200498',
    headers = {'Accept': 'application/json'}
)

# Turn the response's JSON data into a Python dictionary
data = response.json()

# Print URI and name of the Erfgoedobject
print(f"{data['uri']}: {data['naam']}")
# Print a short description of an object
print(f"{data['korte_beschrijving']}")

print(f"Designations:")
# Loop through all the relations
for r in data['relaties']:
  # Only use the relations with aanduidingsobjecten
  if r['verwant']['id'] == 5:
    # Print the type of relation, type of designation, URI of the object, name of the object
    print(f"* {r['relatietype']['naam']} {r['aanduidingsobjecttype']} {r['uri']}: {r['omschrijving']}")

    # Make a request and store the response
    aanduidingsobjectresponse = requests.get(
      r['uri'],
      headers = {'Accept': 'application/json'}
    )

    # Turn the response's JSON data into a Python dictionary
    aanduidingsobjectdata = aanduidingsobjectresponse.json()

    # Print information on the related besluiten
    for besluiten in aanduidingsobjectdata['besluiten']:
      print(f"\t {besluiten['datum_ondertekening']} {besluiten['uri']}: {besluiten['onderwerp']}")

#Some polishing

There are a few things we can do to make the previous example a little better. Either more functional, or more performant.

First, we'll look at a slight performance increase. Right now, we make serveral calls to *requests.get*, every call opens a connection to a server and closes it when the call is finished. In this example we make one call for the Erfgoedobject and three calls for the Aanduidingsobjecten. We open and close the connection 4 times. Since all connections are to the same server (inventaris.onroerenderfgoed.be), this is rather redundant. However, it's possible to use a single connection for all the GET requests by using a *requests.Session* object. This way, the connection will be opened once and closed once. The more requests we do in one script, the more performance we gain. It's also possible to set default values in a requests session, so we don't have to specify the same thing over and over again. Eg., we can set the *Accept header* as a default.

In [ ]:
# Import requests library to contact the REST service
import requests
# Import JSON library to pretty print JSON
import json

# Create a requests Session
session = requests.Session()

# Set a default accept header of application/json
# This ensure that we always ask for application/json, 
# unless we specifically override it
session.headers.update({'Accept': 'application/json'})

# Make a request and store the response
response = session.get(
    'https://id.erfgoed.net/erfgoedobjecten/200498'
)

# Turn the response's JSON data into a Python dictionary
data = response.json()

# Print URI and name of the Erfgoedobject
print(f"{data['uri']}: {data['naam']}")
# Print a short description of an object
print(f"{data['korte_beschrijving']}")

print(f"Designations:")
# Loop through all the relations
for r in data['relaties']:
  # Only use the relations with aanduidingsobjecten
  if r['verwant']['id'] == 5:
    # Print the type of relation, type of designation, URI of the object, name of the object
    print(f"* {r['relatietype']['naam']} {r['aanduidingsobjecttype']} {r['uri']}: {r['omschrijving']}")

    # Make a request and store the response
    aanduidingsobjectresponse = session.get(
      r['uri']
    )

    # Turn the response's JSON data into a Python dictionary
    aanduidingsobjectdata = aanduidingsobjectresponse.json()

    # Print information on the related besluiten
    for besluiten in aanduidingsobjectdata['besluiten']:
      print(f"\t {besluiten['datum_ondertekening']} {besluiten['uri']}: {besluiten['onderwerp']}")

Another improvement we can make is to make the output a bit more agreeable on the eyes. Right now we are just printing to the screen, but some markup would be nice. In a command line script, we could do this by writing our output to a file using a markup language. The most well-know markup language is undoubtedly [Markdown](https://daringfireball.net/projects/markdown/). It allows you to write simple text files that can quickly be turned into  easy to read pdf's with a few simple tools. Since we're not on a filesystem here, we'll just print the Markdown directly to our screen.

In [ ]:
# Import IPython library to produce Markdown
import IPython

output = ''
output += '#Main title\n'
output += '*Paragraph 1.* Lorem ipsum dolor sit amet, consectetur adipiscing elit. Aliquam feugiat mollis dui, ac auctor ex. Morbi augue ligula, sodales eu egestas at, aliquam eu mauris. Aenean convallis mi nec massa imperdiet, eu semper lacus porttitor. Aenean eget sagittis lacus. Morbi a augue quis est laoreet malesuada eu vitae purus. Morbi sed eros lacus. Etiam ullamcorper ex id pharetra tempus. Donec viverra, sapien in pretium ullamcorper, massa ligula accumsan massa, et porttitor enim velit ac risus. Morbi viverra volutpat augue at aliquet. Vivamus non tincidunt ligula, non tempus nisl. Morbi quis gravida tortor. Integer blandit metus vitae arcu posuere, in egestas augue vehicula. Pellentesque fringilla, risus in tristique sagittis, dui leo feugiat orci, eu lacinia leo urna eget urna.\n'
output += '\n'
output += '*Paragraph 2.*Vestibulum vehicula scelerisque varius. Quisque lectus enim, semper id ante sit amet, finibus venenatis erat. Aenean sapien libero, feugiat sit amet lacus gravida, vestibulum semper tortor. Vivamus nec euismod diam, ac accumsan metus. Nulla blandit nunc sit amet enim sodales, vitae suscipit mi dignissim. Lorem ipsum dolor sit amet, consectetur adipiscing elit. Sed vitae purus ex. Duis ut euismod lectus. Proin ut nulla id enim eleifend fermentum quis vel ipsum. Morbi maximus et odio sit amet elementum. Suspendisse rhoncus, augue eget cursus rhoncus, metus leo aliquet mi, eget dapibus neque sapien nec urna. Praesent faucibus porttitor sapien ut finibus. Vivamus scelerisque leo risus, eget convallis arcu tincidunt vitae. Phasellus convallis semper varius. Ut tempus ex id blandit tincidunt.\n\n'
output += '**A list**\n'
output += '* Item 1\n'
output += '* Item 2\n'
output += '\n'
output += '##Merry Christmas!\n'
output += '![](https://beeldbank.onroerenderfgoed.be/images/409414/content/medium)\n'

IPython.display.Markdown(output)

As you can see, using Markdown even allows us to visualise images in our browsers. These come from the [Beeldbank Onroerend Erfgoed](https://beeldbank.onroerenderfgoed.be). The Erfgoedobjecten of the Inventaris Onroerend Erfgoed link to images in the Beeldbank. Multiple images in the Beeldbank can link to the same Erfgoedobject. But every Erfgoedobject has exactly one primary image. This image is generally a good illustration of the Erfgoedobject we're dealing with. Since we can now visualise images through Markdown, we'll the primary images of every Erfgoedobject to our script as well.

#Final script

In [ ]:
# Import requests library to contact the REST service
import requests
# Import IPython library to produce Markdown
import IPython

# Create a requests Session
session = requests.Session()

# Set a default accept header of application/json
# This ensure that we always ask for application/json, 
# unless we specifically override it
session.headers.update({'Accept': 'application/json'})

# Make a request and store the response
response = session.get(
    'https://id.erfgoed.net/erfgoedobjecten/200498'
)

# Turn the response's JSON data into a Python dictionary
data = response.json()

primary_image_uri = data['primaire_foto']

image_response = session.get(
    primary_image_uri
)
image_data = image_response.json()

small_image_url = [i['url'] for i in image_data['storage']['images'] if i['object_key'] == 'small'][0]

# Initialise a buffer to hold our output
output = ''
# Print URI and name of the Erfgoedobject
output += f"#{data['naam']} ({data['uri']})\n"
# Print the small image url and an alternate description
output += f"![{image_data['title']}]({small_image_url})\n\n"
# Print a short description of an object
output += f"{data['korte_beschrijving']}\n"

output += f"##Rechtsgevolgen:\n"
# Loop through all the relations
for r in data['relaties']:
  # Only use the relations with aanduidingsobjecten
  if r['verwant']['id'] == 5:
    # Print the type of relation, type of designation, URI of the object, name of the object
    output += f"* *{r['relatietype']['naam']}* **{r['aanduidingsobjecttype']}** {r['omschrijving']} ({r['uri']})\n"

    # Get the full aanduidingsobject
    aanduidingsobject_response = session.get(
      r['uri']
    )

    # Turn the response's JSON data into a Python dictionary
    aanduidingsobject_data = aanduidingsobject_response.json()

    # Print information on the related besluiten
    for besluiten in aanduidingsobject_data['besluiten']:
      output += f"\t* {besluiten['onderwerp']} van *{besluiten['datum_ondertekening']}* ({besluiten['uri']}) \n"

#Display the generated Mardown output
IPython.display.Markdown(output)

# Conclusion

This first notebook teaches how to request individual resources, everything identified by a single URL. We've seen how to request other related resources and how to generate output in Markdown format. We now have a quick way of generating our own personal reports for an Erfgoedobject. The same principles we applied to Erfgoedobjecten, can be applied to other types of Flanders Heritage resource such as Aanduidingsobjecten or Waarnemingen or any other type not mentioned before.

This notebook only talks about single resources, but quite often we're interested in a collection of resources. We're looking for information on all Erfgoedobjecten of type `church` or all Erfgoedobjecten in a certain province or municipality. Or a combination of those two. [The next notebook](https://github.com/OnroerendErfgoed/scriptorium/blob/main/notebooks/02_get_collection.ipynb) deals with accessing these collections of resources and filtering or searching them.